In [ ]:
!pip install pandas
!pip install requests
!pip install bs4

In [10]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from requests.exceptions import ChunkedEncodingError
import re


In [ ]:
url = "https://ideasai.com/"  # Sustituye esto por la URL de la página que quieres scrapear
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')


In [ ]:
# Ahora puedes usar 'soup' para encontrar elementos en la página. Por ejemplo:
titulo = soup.find('title').text # Esto obtendrá el título de la página
titulo

In [ ]:
# All .tag-link inside <div class="tag-container">
tags = soup.find('div', attrs={'class': 'tag-container'}).find_all('a', attrs={'class': 'tag-link'})

tags_list = []

for tag in tags:
    tags_list.append(tag.text)
    
    num_tags = len(tags_list)
    
print(f"El número de etiquetas es: {num_tags}")

In [ ]:
# Enter in one of the tags and get h3 classname .idea
tag = tags_list[0]
tag_url = url + tag
tag_url

response = requests.get(tag_url)
soup = BeautifulSoup(response.text, 'html.parser')

ideas = soup.find_all('h3', attrs={'class': 'idea'})

ideas_list = []

for idea in ideas:
    ideas_list.append(idea.text)
    num_ideas = len(ideas_list)

print(f"El número de ideas es: {num_ideas}")

In [ ]:
# create an empty list to store the data
data = []

# iterate over all the tags
for i, tag in enumerate(tags_list, 1):
    try:
        tag_url = url + tag
        response = requests.get(tag_url)
        response.raise_for_status()  # verifica si hay errores en la respuesta HTTP
        soup = BeautifulSoup(response.text, 'html.parser')
        ideas = soup.find_all('h3', attrs={'class': 'idea'})
        for idea in ideas:
            data.append({'Tag': tag, 'Idea': idea.text})
        print(f"Enlaces procesados: {i}, Enlaces restantes: {len(tags_list) - i}")
    except ChunkedEncodingError as e:
        print(f"Error: {e}. Reintentando...")

# create a dataframe from the list
df = pd.DataFrame(data)

# save the results into a CSV file
df.to_csv('ideas.csv', index=False)

In [15]:
# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('ideas.csv')

# Limpiar la columna 'Idea' usando expresiones regulares
df['Tag'] = df['Tag'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# Guardar el DataFrame modificado en un nuevo archivo CSV
df.to_csv('ideas_limpias.csv', index=False)

In [16]:
# Cargar el archivo CSV modificado en un DataFrame
df_modificado = pd.read_csv('ideas_limpias.csv')

# Imprimir el DataFrame
df_modificado.head()

,Tag,Idea
0,Platform Startup Ideas,A startup building an online platform for recr...
1,Platform Startup Ideas,An online marketplace for tutors for students ...
2,Platform Startup Ideas,A home energy management system that maintains...
3,Platform Startup Ideas,A web-based system for managing and tracking t...
4,Platform Startup Ideas,"The app, which is for kids ages 4-8, helps kid..."


In [17]:
# Convert CSV to JSON
df.to_json('ideas_limpias.json', orient='records')

# Read the JSON file
df = pd.read_json('ideas_limpias.json')
